# Handling trajectory data files (reading & writing)

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas/master?filepath=tutorials/2-reading-data-from-files.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/anitagraser/movingpandas-examples.


### You can either loging with **hf_login.sh** or login with down below code
make sure you have a token created in huggingface, also you should set the token with **write** access

In [1]:
from huggingface_hub import login, whoami
login()

### use the hf_tools python package for download or upload csv files

In [1]:
import numpy as np
import os
import hf_tools.scripts.add_path
from hf_tools.src.hf_tools.hf_sync import *

In [2]:
download_file(file_path='hf_tools/data/', 
                  repo_id="ARG-NCTU/uav-usv-traj", 
                  repo_type="dataset", 
                  hf_path='real_bag_0312_1238_pd_2.csv')

In [3]:
import pandas as pd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")

/home/arg/miniconda3/envs/movingpandas/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)



MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
executable : /home/arg/miniconda3/envs/movingpandas/bin/python
machine    : Linux-5.15.0-92-generic-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.8.4
GDAL data dir: /home/arg/miniconda3/envs/movingpandas/share/gdal
PROJ       : 9.3.1
PROJ data dir: /home/arg/miniconda3/envs/movingpandas/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.2.1
fiona      : 1.9.6
numpy      : 1.26.4
shapely    : 2.0.3
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.8.3
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : 1.2


## Reading CSVs

In [4]:
hvplot_defaults = {'tiles':'EsriImagery', 'frame_height':400, 'frame_width':700, 'cmap':'Viridis', 'colorbar':True}

In [5]:
%%time 
df = pd.read_csv('hf_tools/data/real_bag_0312_1238_pd_2.csv', delimiter=',')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='Y', y='X')
traj_collection.hvplot(title=str(traj_collection), line_width=3, **hvplot_defaults)

CPU times: user 482 ms, sys: 15.7 ms, total: 498 ms
Wall time: 497 ms


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (triangle_angle)

## Reading MovingFeatures JSONs

In [ ]:
%%time 
file_name = 'data/movingfeatures.json'
traj = mpd.read_mf_json(file_name)
traj

In [ ]:
traj.plot()

In [ ]:
traj.hvplot(title='Wind measure along trajectory', c='wind', line_width=5, **hvplot_defaults)

## Writing

### as points

In [ ]:
point_gdf = traj_collection.to_point_gdf()
point_gdf.head()

In [ ]:
point_gdf.to_file("temp.gpkg", layer='points', driver="GPKG")
read_file('temp.gpkg', layer='points').plot()

### as lines

In [ ]:
line_gdf = traj_collection.to_line_gdf()
line_gdf.head()

In [ ]:
line_gdf.to_file("temp.gpkg", layer='lines', driver="GPKG")
read_file('temp.gpkg', layer='lines').plot()

### as trajectories

In [ ]:
traj_gdf = traj_collection.to_traj_gdf(wkt=True)
traj_gdf

In [ ]:
traj_gdf.to_file("temp.gpkg", layer='trajectories', driver="GPKG")
read_file('temp.gpkg', layer='trajectories').plot()

## Error messages while reading

The following errors are expected:

### Missing datetime info

In [ ]:
gdf = read_file('data/demodata_geolife.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')

### Missing geometry info

In [ ]:
df = pd.read_csv('data/demodata_geolife.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t')

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Handling trajectory data files (reading & writing)](2-reading-data-from-files.ipynb)
1. [TrajectoryCollection aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)
1. [Working with local coordinates](5-local-coordinates.ipynb)